In [ ]:
import tdmclient
import tdmclient.notebook
await tdmclient.notebook.start()


# 1m x 1m paper: 200 x 200 coordinates

# Information needed from the other parts:

# Camera measurement = y_t = np.array([x_position, y_position])
# Camera measurement of robot angle = theta

# Global path = [(x_pos1, y_pos1), (x_pos2, y_pos2), ... (x_posn, y_posn)]
global_path = path # See: global navigation part

import math
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
@tdmclient.notebook.sync_var
def motors(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    return motor_left_target, motor_right_target


In [ ]:
def predict(state, input_val, covariance, Ts, theta, thymio_b=11, thymio_r=2):
    # Prediction step
    A = np.array([[1, 0, Ts, 0],
                  [0, 1, 0, Ts],
                  [0, 0, 1,  0],
                  [0, 0, 0,  1]])

    B = np.array([[0.5*math.cos(theta)*Ts, 0.5*math.cos(theta)*Ts],
                  [0.5*math.sin(theta)*Ts, 0.5*math.sin(theta)*Ts],
                  [Ts/thymio_b, -Ts/thymio_b],
                  [-1/thymio_r, 1/thymio_r]])
    

    new_state = np.dot(A, state) + np.dot(B, input_val)
    new_covariance = np.dot(A, np.dot(covariance, A.T)) + process_noise

    return new_state, new_covariance


def measure(state, covariance, cam_measurement):
    # Update step based on camera measurement
    if cam_measurement = ...: # camera not covered
        C = np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0]])  # Measurement matrix

        R = measurement_noise # Measurement noise covariance
        i_t = cam_measurement - np.dot(C, state) #innovation 
        S_t = np.dot(C, np.dot(covariance, C.T)) + R #innovation of variance
    
        K = np.dot(covariance, np.dot(C.T, np.linalg.inv(S_t))) #gain
    
        new_state = state + np.dot(K, i_t)
        new_covariance = np.dot((np.eye(2) - np.dot(K, C)), covariance)
    else: # camera covered
        new_state = state
        new_covariance = covariance
    
    return new_state, new_covariance

In [ ]:
def turn_to_target(pos_1, pos_2, thymio_angle): #turn towards the target when the angle difference is bigger than 10°, since otherwise the path will be inaccurate
    global motor_left_target, motor_right_target
    x_dist = pos_2[0] - pos_1[0]
    y_dist = pos_2[1] - pos_1[1]
    target_angle = math.atan2(y_dist, x_dist)/math.pi*180
    angle_diff = (target_angle - thymio_angle + 180) % 360 - 180  # result always between -180° and 180°, positive when target angle is greater than thymio angle
    if abs(angle_diff) > 10:
        if angle_diff > 0:
            motors(-50,50)
            time.sleep(0.05*angle_diff)
            motors(0,0)
        else:
            motors(50,-50)
            time.sleep(0.05*angle_diff)
            motors(0,0)


def go_to_target(pos_1, pos_2, thymio_angle, pos_gain = 10, angle_gain = 0.85):#code to bring Thymio back to the right direction
    global motor_left_target, motor_right_target
    x_dist = pos_2[0] - pos_1[0]
    y_dist = pos_2[1] - pos_1[1]
    target_angle = math.atan2(y_dist, x_dist)/math.pi*180
    control_x = pos_gain * x_dist
    control_y = pos_gain * y_dist
    angle_diff = (target_angle - thymio_angle + 180) % 360 - 180  # result always between -180° and 180°, positive when target angle is greater than thymio angle
    control_angle = angle_gain * angle_diff
    motor_left_target = math.ceil(control_x - control_angle)
    motor_right_target = math.ceil(control_x + control_angle)
    
    return motor_left_target, motor_right_target


In [ ]:
%%run_python

#initial variables

# Kalman filter parameters

#from camera/global path
x_pos = global_path[0][0]
y_pos = global_path[0][1]
initial_state = np.array([x_pos, y_pos, 0.0, 0.0])  # Initial state: [position_x, position_y, linear velocity = 0, angular velocity = 0]

#to be changed experimentally with Thymio
initial_covariance = np.diag([1.0, 1.0, 1.0, 1.0])  # Initial covariance matrix
process_noise = np.diag([0.03, 0.03, 0.01, 0.01])  # Process noise covariance matrix
measurement_noise = np.array([0.1, 0.1])  # Measurement noise variance

# Initialize Kalman filter
state = initial_state
covariance = initial_covariance
Ts = 0.5
motor_left_target = 0
motor_right_target = 0
cor = 2 #correction factor for the target coordinates (1cm)

# For plot:
predicted_positions = []
cam_measured_positions = []
estimated_positions = []

# Stores the motor values
motor_values = []

# Control loop: main code
for coordinate in global_path:
    #while tup(state[0],state[1]) != coordinate:
    while (state[0]>(coordinate[0]+cor) or state[0]<(coordinate[0]-cor)) and (state[1]>(coordinate[1]+cor) or state[0]<(coordinate[1]-cor)):
        theta = ... # to be calculated wih camera
        #cam_measurement = get_cam_measurement()
        cam_measured_positions.append(cam_measurement)
        
        left_speed = motor_left_target
        right_speed = motor_right_target
        input_val = np.array([left_speed, right_speed])
        
        state, covariance = predict(state, input_val, covariance, Ts, theta)
        predicted_positions.append(state[0:2].tolist())
        
        state, covariance = measure(state, covariance, cam_measurement)
        estimated_positions.append(state[0:2].tolist())
        
        turn_to_target(tup(state[0],state[1]), coordinate, theta)
        motor_left_target, motor_right_target = go_to_target(tup(state[0],state[1]), coordinate, theta)
        motor_values.append(motor_left_target, motor_right_target)


# Plot the results:

# values for x:
#sampling_times = np.linspace(0, Ts*len(predicted_positions), Ts)
predicted_x = [position[0] for position in predicted_positions]
predicted_y = [position[1] for position in predicted_positions]

cam_x = [position[0] for position in cam_measured_positions]
cam_y = [position[1] for position in cam_measured_positions]

estimated_x = [position[0] for position in estimated_positions]
estimated_y = [position[1] for position in estimated_positions]

# plot (x- and y-position):
plt.plot(predicted_x, predicted_y, label='Predicted positions')
plt.plot(cam_x, cam_y, label='Camera measured positions')
plt.plot(estimated_x, estimated_y, label='Estimated/corrected positions:')


plt.xlabel('x-positions')
plt.ylabel('y-positions')
plt.title('Kalman filter results')
plt.legend()
plt.show()